In [253]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GridSearchCV
import seaborn as sns

In [254]:
# Load the data
nordpool = pd.read_csv('New Data/2021/Nordpool DKK.csv')
energinet = pd.read_csv('New Data/2021/Energinet DKK.csv')
weather_observation = pd.read_csv('New Data/2021/Weather Observation.csv')
weather_forecast = pd.read_csv('New Data/2021/Weather Forecast.csv')
network = pd.read_csv('New Data/2021/Network Manager 21.csv')

# Filter energinet price area
energinet = energinet[energinet['PriceArea | PriceArea | 804696'] == 'DK2']
# Drop price area
energinet.drop('PriceArea | PriceArea | 804696', axis=1, inplace=True)

# Convert ts to datetime
nordpool['ts'] = pd.to_datetime(nordpool['ts'])
energinet['ts'] = pd.to_datetime(energinet['ts'])
weather_observation['ts'] = pd.to_datetime(weather_observation['ts'])
weather_forecast['ts'] = pd.to_datetime(weather_forecast['ts'])
network['ts'] = pd.to_datetime(network['ts'])

# Resample network_manager data to hourly intervals (average the data)
network = network.set_index('ts')
network = network.resample('h').mean()
network.reset_index(inplace=True)

In [255]:
network.head()

,ts,Hasle Common 10kV Lines Voltage | has_fel_10kvskinnespend | 804133,Hasle Vind Active Power | has_vin_effekt | 804123,Hasle Vind Current | has_vin_belastning | 804117
0,2021-01-01 00:00:00,10.505372,0.310046,19.138536
1,2021-01-01 01:00:00,10.499852,0.739963,42.792531
2,2021-01-01 02:00:00,10.440138,0.877331,50.523536
3,2021-01-01 03:00:00,10.423414,0.842384,47.936477
4,2021-01-01 04:00:00,10.401766,0.469178,28.816449


## Step 2: Data Handling

In [256]:
# Compare length of dfs + get first & last timestamp
def get_first_last(**dfs):
    # Initialize
    first_last = pd.DataFrame(index=['first', 'last', 'length', 'start_check', 'end_check'])
    col_name='ts'
    # Save first and last timestamp for each dataframe
    for name, df in dfs.items():
        first_ts = df['ts'].iloc[0]
        last_ts = df['ts'].iloc[-1]
        start_check = start == first_ts
        end_check = end == last_ts
        first_last[name] = [first_ts, last_ts, len(df), start_check, end_check]
    return first_last

# Initialize
start = pd.to_datetime('2021-01-01 00:00:00')
end = pd.to_datetime('2021-12-31 23:00:00')

print("Length of a year in hours:", 365*24)
get_first_last(nordpool=nordpool, energinet=energinet, weather_observation=weather_observation, weather_forecast=weather_forecast, network=network)

Length of a year in hours: 8760


,nordpool,energinet,weather_observation,weather_forecast,network
first,2021-01-02 00:00:00,2021-01-01 00:00:00,2021-01-15 16:00:00,2021-01-01 00:00:00,2021-01-01 00:00:00
last,2021-12-31 23:00:00,2021-12-31 23:00:00,2021-12-31 23:00:00,2021-12-29 11:00:00,2021-12-31 23:00:00
length,8735,8759,7936,8694,8760
start_check,False,True,False,True,True
end_check,True,True,True,False,True


### Data Merging

In [257]:
# Merge the dataframes on network with left join
data = pd.merge(network, weather_forecast, on='ts', how='left')
data = pd.merge(data, weather_observation, on='ts', how='left') 

# Sort the data by the timestamp column 'ts' to ensure time order
data21 = data.sort_values(by='ts')

# Save to csv
# data21.to_csv('New Data/2021/Wind Data 21.csv', index=False)

data21.head()

,ts,Hasle Common 10kV Lines Voltage | has_fel_10kvskinnespend | 804133,Hasle Vind Active Power | has_vin_effekt | 804123,Hasle Vind Current | has_vin_belastning | 804117,Weather forecast for the max air temperature for the coming 6 hours at Bornholm | 9F7P/00/00/MET-Norway/forecast/air_temperature_max | 128206,Weather forecast for wind direction at Bornholm | 9F7P/00/00/MET-Norway/forecast/wind_direction | 128270,Weather forecast for the precipitation amount for the coming 1 hour at Bornholm | 9F7P/00/00/MET-Norway/forecast/precipitation_amount_acc | 128238,Weather forecast for the relativehumidity amount at Bornholm | 9F7P/00/00/MET-Norway/forecast/relative_humidity_2m | 128254,Weather forecast for wind speed at Bornholm | 9F7P/00/00/MET-Norway/forecast/wind_speed | 128286,Weather forecast for wind speed y_direction at Bornholm | 9F7P/00/00/MET-Norway/forecast/y_wind_10m | 128318,...,Weather forecast for wind speed x_direction at Bornholm | 9F7P/00/00/MET-Norway/forecast/x_wind_10m | 128302,Weather forecast for the air temperature at Bornholm | 9F7P/00/00/MET-Norway/forecast/air_temperature_2m | 128190,Observed maximum temperature past hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/temp_max_past1h/06193 | 406592,Observed mean intensity of global radiation in the latest hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/radia_glob_past1h/06193 | 406672,Observed minimum temperature past hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/temp_min_past1h/06193 | 406608,Observed mean wind speed the past hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/wind_speed_past1h/06193 | 406640,Observed mean humidity past hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/humidity_past1h/06193 | 406576,Observed mean temperature past hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/temp_mean_past1h/06193 | 406560,Observed mean wind direction the past hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/wind_dir_past1h/06193 | 406624,Observed accumulated precipitation the past hour at Hammer Odde Fyr - DMI station 06193 | 9F7P/7Q/XC/DMI/metObs/precip_past1h/06193 | 406656
0,2021-01-01 00:00:00,10.505372,0.310046,19.138536,275.52020,189.13286,0.000000,0.848557,3.711973,3.658367,...,0.593647,275.51404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-01 01:00:00,10.499852,0.739963,42.792531,275.50630,191.73468,0.269043,0.937904,3.358219,3.176009,...,0.794102,275.36557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-01 02:00:00,10.440138,0.877331,50.523536,275.13600,184.39082,0.167969,0.974747,3.255157,3.315285,...,0.328136,275.08374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-01 03:00:00,10.423414,0.842384,47.936477,274.88626,178.39856,0.000000,0.973976,3.109769,3.461883,...,-0.328434,274.86594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-01 04:00:00,10.401766,0.469178,28.816449,274.93670,170.38286,0.662842,0.989250,3.090981,3.048118,...,-0.469315,274.80240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Load merged data from 2022  (neccessary for feature engineering like lags)

In [258]:
# Load csv
data22 = pd.read_csv('New Data/2022/Wind Data 22.csv')

# Concatenate the dataframes
data = pd.concat([data21, data22], join='inner', axis=0, ignore_index=True)

# Ensure datetime format
data['ts'] = pd.to_datetime(data['ts'])

Column cleaning

In [259]:

# Use regex to remove everything starting from '|'
data.columns = data.columns.str.replace(r'\|.*', '', regex=True)
nordpool.columns = nordpool.columns.str.replace(r'\|.*', '', regex=True)
energinet.columns = energinet.columns.str.replace(r'\|.*', '', regex=True)

# Use regex to remove everything starting from the word "at"
data.columns = data.columns.str.replace(r'\sat.*', '', regex=True)
data.columns = data.columns.str.replace(r'Hasle', '', regex=True)
data.columns = data.columns.str.replace(r'Vind', 'Wind', regex=True)

# Strip columns
data.columns = data.columns.str.strip()

data.columns

Index(['ts', 'Common 10kV Lines Voltage', 'Wind Active Power', 'Wind Current',
       'Weather forecast for the max air temperature for the coming 6 hours',
       'Weather forecast for wind direction',
       'Weather forecast for the precipitation amount for the coming 1 hour',
       'Weather forecast for the relativehumidity amount',
       'Weather forecast for wind speed',
       'Weather forecast for wind speed y_direction',
       'Weather forecast for solar shortwave flux',
       'Weather forecast for the minimum air temperature for the coming 6 hours',
       'Weather forecast for wind speed x_direction',
       'Weather forecast for the air temperature',
       'Observed maximum temperature past hour',
       'Observed mean intensity of global radiation in the latest hour',
       'Observed minimum temperature past hour',
       'Observed mean wind speed the past hour',
       'Observed mean humidity past hour',
       'Observed mean temperature past hour',
       'Observed

In [260]:
# Resample the data by months ('M') and count missing values for each period
data = data.set_index('ts')
missing_per_month = data.isnull().resample('M').sum()
# Reset
data = data.reset_index()

# Convert the resampled data into a long-form dataframe for easier plotting with Seaborn
missing_per_month_long = missing_per_month.reset_index().melt(id_vars='ts', var_name='Feature', value_name='Missing Count')

# Create the seaborn barplot
plt.figure(figsize=(15, 6))
sns.barplot(x='ts', y='Missing Count', hue='Feature', data=missing_per_month_long)
plt.xticks(rotation=60)
plt.title('Missing Value Counts per Month')
plt.xlabel('Month')
plt.ylabel('Missing Value Count')
plt.legend(title='Feature', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

<Figure size 1500x600 with 0 Axes>

### Missing Values Handling

In [ ]:
# df_interpolated = data.interpolate(method='linear')

# Set ts as index
missing_before = data.isnull().sum()
data = data.set_index('ts')
# Interpolate missing values (time series-based)
data = data.interpolate(method='time')
# Reset index
data = data.reset_index()
missing_after = data.isnull().sum()

missing_summary = pd.DataFrame({'Before': missing_before, 'After': missing_after})
missing_summary

Features for Weather observation can not be interpolated as they occur in the very beginning. (01/01/21 - 15/01/21)

### Feature Engineering

- Lagged days
- Quantiles from prev week
- Wind power production is proportional to the cube of the wind speed: $P = \frac{1}{2} \rho \pi r^2 V^3 $
- Time-based features like day and hour


Since 'Observed mean wind speed the past hour' and 'Observed Wind Speed Cubed' are highly correlated, we may encounter multicollinearity which can inflate variance estimates for regression coefficients. 

-> Lasso or Ridge regression or Nonlinear Regression needed?


In [ ]:
data = data.rename(columns={'Wind Active Power': 'Wind Power'})

# Shift by 24 hours for daily data
data['Wind Power Lag1'] = data['Wind Power'].shift(24)
data['Wind Power Lag2'] = data['Wind Power'].shift(48)
data['Wind Power Lag3'] = data['Wind Power'].shift(72)

# Rolling average for 1 week (=168h) for several quantiles
data['Wind Power 0.50 Quantile'] = data['Wind Power'].rolling(window=168).quantile(0.5)
data['Wind Power 0.75 Quantile'] = data['Wind Power'].rolling(window=168).quantile(0.75)
data['Wind Power 0.95 Quantile'] = data['Wind Power'].rolling(window=168).quantile(0.95)

# Cube of windspeed? (to capture non-linear relationships or wait for polynomial features?)
data['Observed Wind Speed Cubed'] = data['Observed mean wind speed the past hour']**3
data['Forecasted Wind Speed Cubed'] = data['Weather forecast for wind speed']**3

# Time-based features
data['Day'] = data['ts'].dt.dayofweek
data['Hour'] = data['ts'].dt.hour
data['Month'] = data['ts'].dt.month

Cut down the data as weather_observation starts **15/01/2021** and ends **31/12/2021**?

In [ ]:
# Exclude all before 15/01/2021 for data
start = pd.to_datetime('2021-01-15 16:00:00')
data = data[data['ts'] >= start]
# Exclude all after 31/12/2021 for data
data = data[data['ts'] <= end]
# Reset index
data = data.reset_index(drop=True)
# Check success
len(data)

In [ ]:
# Check missing values
data.isnull().sum()

In [ ]:
# # Show me the missing values with ts
# missing_values = data[data.isnull().any(axis=1)]
# missing_values = missing_values[['ts', 'Observed mean wind speed the past hour']]
# missing_values

# # Drop rows with missing values
# data = data.dropna()

# # Adjust the index
# data = data.reset_index(drop=True)

### Data Visualization

In [ ]:
# Correlation matrix between features
corr_matrix = data.corr()
# print(corr_matrix)

plt.figure(figsize=(15, 12))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', center=0, linewidths=0.5, 
            cbar_kws={"shrink": 0.75}, square=True)
plt.title('Correlation Matrix Heatmap', fontsize=18)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()

plt.show()

In [ ]:
palette1 = sns.color_palette("pastel")
palette2 = sns.color_palette("bright")

# Plot moving average of wind power vs time
plt.figure(figsize=(15, 6))
sns.lineplot(x=data['ts'], y=data['Wind Power'], label='Wind Power', color=palette1[0])
sns.lineplot(x=data['ts'], y=data['Wind Power'].rolling(window=168).mean(), label='1 Week Moving Average', color=palette2[1])
sns.lineplot(x=data['ts'], y=data['Wind Power'].rolling(window=336).mean(), label='2 Weeks Moving Average', color=palette2[2])
sns.lineplot(x=data['ts'], y=data['Wind Power'].rolling(window=672).mean(), label='4 Weeks Moving Average', color=palette2[3])
plt.title('Wind Power vs Time')
plt.xlabel('Time')
plt.ylabel('Wind Power')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Plot wind power vs wind current
plt.figure(figsize=(12, 6))
sns.scatterplot(x='Wind Current', y='Wind Power', data=data, alpha=0.3)
plt.plot([0, data['Wind Current'].max()], [0, data['Wind Power'].max()], color='red', linestyle='--')
plt.title('Wind Power vs Wind Current')
plt.xlabel('Wind Current')
plt.ylabel('Wind Power')
plt.show()

In [ ]:
# Plot wind power vs weather forecast for wind speed
plt.figure(figsize=(12, 6))
sns.scatterplot(x='Weather forecast for wind speed', y='Wind Power', data=data, alpha=0.3)
plt.plot([0, data['Weather forecast for wind speed'].max()], [0, data['Wind Power'].max()], color='red', linestyle='--')
plt.title('Wind Power vs Weather Forecast for Wind Speed')
plt.xlabel('Forecast mean wind speed')
plt.ylabel('Wind Power')
plt.show()

## Step 3: Linear Regression

Split 80/20 train and test

In [ ]:
# Define X, y
X = data.drop(columns=['ts', 'Wind Power', 'Wind Current']) # Features
ts = data['ts']
y = data['Wind Power']

# Split 80/20
split_index = int(len(X) * 0.8)
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert scaled arrays back to DataFrames with original indices and column names
X_train = pd.DataFrame(X_train, index=X.index[:split_index], columns=X.columns)
X_test = pd.DataFrame(X_test, index=X.index[split_index:], columns=X.columns)
y_train = pd.Series(y_train.values, index=y.index[:split_index])
y_test = pd.Series(y_test.values, index=y.index[split_index:])

# Confirm index alignment
print("Train Index Equality:", X_train.index.equals(y_train.index))
print("Test Index Equality:", X_test.index.equals(y_test.index))

# Print shapes of the resulting datasets
print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

X2 = X including 'Wind Current‘ feature

In [ ]:
# Define X2
X2 = data.drop(columns=['ts', 'Wind Power']) # Features

# Split 80/20
X_train2, X_test2 = X2[:split_index], X2[split_index:]

# Scale the features using StandardScaler
X_train2 = scaler.fit_transform(X_train2)
X_test2 = scaler.transform(X_test2)

# Convert scaled arrays back to DataFrames with original indices and column names
X_train2 = pd.DataFrame(X_train2, index=X2.index[:split_index], columns=X2.columns)
X_test2 = pd.DataFrame(X_test2, index=X2.index[split_index:], columns=X2.columns)

# Confirm index alignment
print("Train Index Equality:", X_train2.index.equals(y_train.index))
print("Test Index Equality:", X_test2.index.equals(y_test.index))

# Print shapes of the resulting datasets
print(f"Training data shape: {X_train2.shape}")
print(f"Testing data shape: {X_test2.shape}")

In [ ]:
# # Define X, y
# X = data.drop(columns=['ts', 'Wind Power']) # Features
# ts = data['ts']
# y = data['Wind Power']

# # ts_test = ts_column[X_test.index] # if neccessary later

# # Scale the features using StandardScaler
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)

# # Get shapes
# print("Train set:", X_train.shape)
# print("Test set:", X_test.shape)

# # Check indices
# X_train = pd.DataFrame(X_train)
# y_train = pd.Series(y_train)
# X_test = pd.DataFrame(X_test)
# y_test = pd.Series(y_test)

# print("Train Index Equality:", X_train.index.equals(y_train.index))
# print("Test Index Equality:", X_test.index.equals(y_test.index))

# # # Align indices
# # X_test = pd.DataFrame(X_test, columns=X.columns)
# # y_test = pd.Series(y_test.values, index=y_test.index)
# # X_test.index = y_test.index

# print("Test Index Equality (2):", X_test.index.equals(y_test.index))

In [ ]:
# Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

### Step 3.1: Linear Regression (small sample)

Implementation of both approaches

In [ ]:
# Step 3.1: Gradient Descent
class LinearRegressionGD:

    # attributes of the class
    def __init__(self, learning_rate=0.01, n_iterations=1000): 
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.theta = None  # Parameter vector (weights and bias)
        self.cost_history = []  # To store cost function values over iterations
    
    # gradient descent algorithm
    def fit(self, X, y): 
        # Ensure that X and y are NumPy arrays and convert them to float type
        X = np.array(X, dtype=float)
        y = np.array(y, dtype=float)

        # Number of training samples (m) and number of features (n)
        m, n = X.shape

        # Add a bias (intercept) column of ones to the input matrix X
        X_b = np.c_[np.ones((m, 1)), X]  # X_b is now (m, n+1) with the bias term
        
        # Initialize theta (parameters) with zeros
        self.theta = np.zeros(n + 1)  # Including bias
        
        # Gradient Descent loop
        for _ in range(self.n_iterations):
            # Predicted values (y_hat = X_b * theta)
            y_pred = X_b.dot(self.theta)
            
            # Compute cost (MSE)
            cost = (1 / (2 * m)) * np.sum((y_pred - y) ** 2)
            self.cost_history.append(cost)  # Store cost for plotting

            # Gradient (in matrix form)
            gradient = (1 / m) * X_b.T.dot(y_pred - y)
            
            # Update the parameters theta
            self.theta -= self.learning_rate * gradient
    
    # prediction function
    def predict(self, X):
        # Ensure that X is a NumPy array and convert it to float type
        X = np.array(X, dtype=float)
        
        # Add a bias (intercept) column of ones to the input matrix X
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        
        # Return predictions (y_pred = X_b * theta)
        return X_b.dot(self.theta)

################################################################################################################################################

# Step 3.1: Closed-form solution (Normal Equation)
def ClosedFormSolution(X, y):
    # Ensure that X and y are NumPy arrays and convert them to float type
    X = np.array(X, dtype=float)
    y = np.array(y, dtype=float)
    
    # Add a bias (intercept) column of ones to the input matrix X
    X_b = np.c_[np.ones((X.shape[0], 1)), X]
    
    # Normal equation: theta = (X.T * X)^(-1) * X.T * y
    theta_best = np.linalg.pinv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
    
    return theta_best

Test of both approaches on small sample (e.g. 200 rows)

In [ ]:
# Small subset of train data
X_train_small = np.array(X_train[100:300], dtype=float)  
y_train_small = np.array(y_train[100:300], dtype=float)

# Train using Gradient Descent
gd_model = LinearRegressionGD(learning_rate=0.001, n_iterations=1000)
gd_model.fit(X_train_small, y_train_small)
y_pred_gd = gd_model.predict(X_train_small)

# Train using Closed-Form Solution (Normal Equation)
theta_best = ClosedFormSolution(X_train_small, y_train_small)
X_b_small = np.c_[np.ones((X_train_small.shape[0], 1)), X_train_small]
y_pred_closed_form = X_b_small.dot(theta_best)

In [ ]:
# Create a figure with three subplots: Gradient Descent progress, Actual vs. Predicted, Parity plot
fig, axs = plt.subplots(1, 3, figsize=(18, 6)) 

# Subplot 1: Cost function over iterations (for Gradient Descent)
axs[0].plot(gd_model.cost_history, label='Cost Function (MSE) over Iterations', color='b')
axs[0].set_xlabel('Iterations')
axs[0].set_ylabel('Mean Squared Error (MSE)')
axs[0].set_title('Gradient Descent Progress')
axs[0].legend()

# Subplot 2: Actual vs Predicted values for both methods
axs[1].plot(y_train_small, label='Actual Values', marker='o', color='g')
axs[1].plot(y_pred_gd, label='Predicted Values (Gradient Descent)', marker='x', linestyle='--', color='r')
axs[1].plot(y_pred_closed_form, label='Predicted Values (Closed Form)', marker='.', linestyle=':', color='b')
axs[1].set_xlabel('Sample Index')
axs[1].set_ylabel('Target Variable (y)')
axs[1].set_title('Actual vs Predicted Values')
axs[1].legend()

# Subplot 3: Parity plot for both methods (Predicted vs Actual with 45-degree line)
min_val = min(y_train_small.min(), y_pred_gd.min(), y_pred_closed_form.min())
max_val = max(y_train_small.max(), y_pred_gd.max(), y_pred_closed_form.max())

# Add the 45-degree perfect prediction line
axs[2].plot([min_val, max_val], [min_val, max_val], 'k--', label='Perfect Prediction (45° Line)')

# Scatter plot for Gradient Descent predictions
axs[2].scatter(y_train_small, y_pred_gd, label='Gradient Descent', color='r', alpha=0.5)

# Scatter plot for Closed-Form predictions
axs[2].scatter(y_train_small, y_pred_closed_form, label='Closed Form', color='b', alpha=0.5)

axs[2].set_xlabel('Actual Values')
axs[2].set_ylabel('Predicted Values')
axs[2].set_title('Parity Plot (Predicted vs Actual)')
axs[2].legend()

# Adjust layout for better spacing
plt.tight_layout()
plt.show()

# Verify that both solutions are close by comparing the predictions
print("Gradient Descent Coefficients:", gd_model.theta)
print("Closed-Form Solution Coefficients:", theta_best)
# print("Are the predictions close? ", np.allclose(y_pred_gd, y_pred_closed_form, atol=1e-1))

### Step 3.2: Linear Regression (large sample)

In [ ]:
# Closed-Form Solution (Normal Equation)
theta_best_full = ClosedFormSolution(X_train, y_train) # Compute on the train data
X_b_full = np.c_[np.ones((X_test.shape[0], 1)), X_test] # Predictions on the test / unseen data!
y_test_pred_closed_form = X_b_full.dot(theta_best_full)

# Train using Gradient Descent on full dataset
gd_model = LinearRegressionGD(learning_rate=0.00001, n_iterations=500000)  
gd_model.fit(X_train, y_train) # Compute on the train data
y_test_pred_gd = gd_model.predict(X_test) # Predictions on the test / unseen data!

# Convert predictions to pandas Series with y_test's original index
y_test_pred_closed_form = pd.Series(y_test_pred_closed_form, index=y_test.index)
y_test_pred_gd = pd.Series(y_test_pred_gd, index=y_test.index)

In [ ]:
# Create a figure with three subplots side by side
fig, axs = plt.subplots(1, 3, figsize=(18, 6))  # 1 row, 3 columns

# Subplot 1: Cost function over iterations (for Gradient Descent)
axs[0].plot(gd_model.cost_history, label='Cost Function (MSE) over Iterations', color='b')
axs[0].set_xlabel('Iterations')
axs[0].set_ylabel('Mean Squared Error (MSE)')
axs[0].set_title('Gradient Descent Progress')
axs[0].legend()

# Subplot 2: Actual vs Predicted values for both methods
axs[1].plot(y_test[500:1000], label='Actual Values', marker='o', color='g')
axs[1].plot(y_test_pred_gd[500:1000], label='Predicted Values (Gradient Descent)', marker='x', linestyle='--', color='r')
axs[1].plot(y_test_pred_closed_form[500:1000], label='Predicted Values (Closed Form)', marker='.', linestyle=':', color='b')
axs[1].set_xlabel('Sample Index')
axs[1].set_ylabel('Target Variable (y)')
axs[1].set_title('Actual vs Predicted Values (Test Set)')
axs[1].legend()

# Subplot 3: Parity plot for both methods (Predicted vs Actual with 45-degree line)
min_val = min(y_test.min(), y_test_pred_gd.min(), y_test_pred_closed_form.min())
max_val = max(y_test.max(), y_test_pred_gd.max(), y_test_pred_closed_form.max())

# Add the 45-degree perfect prediction line
axs[2].plot([min_val, max_val], [min_val, max_val], label='Perfect Prediction (45° Line)', linestyle='-', linewidth=4 ,color='g')
# Scatter plot for Gradient Descent predictions
axs[2].scatter(y_test, y_test_pred_gd, label='Gradient Descent', color='r', alpha=0.4)
# Scatter plot for Closed-Form predictions
axs[2].scatter(y_test, y_test_pred_closed_form, label='Closed Form', color='b', alpha=0.4)

axs[2].set_xlim(-0.2,9)
axs[2].set_xlabel('Actual Values')
axs[2].set_ylabel('Predicted Values')
axs[2].set_title('Parity Plot (Predicted vs Actual)')
axs[2].legend()

# Adjust layout for better spacing
plt.tight_layout()
plt.show()

# Output the coefficients for both methods
print("Gradient Descent Coefficients:", gd_model.theta)
print("Closed-Form Solution Coefficients:", theta_best_full)

# Compare the predictions and check if they are close
print("Are the predictions close? ", np.allclose(y_test_pred_gd, y_test_pred_closed_form))

In [ ]:
# Step 3.3: Evaluate the models 
rmse_gd = np.sqrt(mean_squared_error(y_test, y_test_pred_gd))
r2_gd = r2_score(y_test, y_test_pred_gd)
print(f'Mean Squared Error (Gradient Descent): {rmse_gd}')
print(f'R-squared (Gradient Descent): {r2_gd}')

rmse_closed_form = np.sqrt(mean_squared_error(y_test, y_test_pred_closed_form))
r2_closed_form = r2_score(y_test, y_test_pred_closed_form)
print(f'Mean Squared Error (Closed Form): {rmse_closed_form}')
print(f'R-squared (Closed Form): {r2_closed_form}')


In [ ]:
# Get coefficients from the Linear Regression model
cols = ['Intercept'] + list(X_train.columns)
lr_coefficients_df = pd.DataFrame({ 'Feature': cols, 'Coefficient (GD)': gd_model.theta, 'Coefficient (Closed Form)': theta_best_full})
lr_coefficients_df.T

In [ ]:
# # Step 3.3: Evaluate the model on the testing dataset
# X_b_test = np.c_[np.ones((X_test.shape[0], 1)), X_test]
# y_test_pred = X_b_test.dot(theta_best_full)

# # Evaluation Metrics
# rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
# mae = mean_absolute_error(y_test, y_test_pred)
# r2 = r2_score(y_test, y_test_pred)

# print(f"Root Mean Squared Error (RMSE): {rmse}")
# print(f"Mean Absolute Error (MAE): {mae}")
# print(f"R-squared (R²): {r2}")

### Step 3.3: Linear Regression (large sample + Wind Current)

In [ ]:
# Closed-Form Solution (Normal Equation)
theta_best_full2 = ClosedFormSolution(X_train2, y_train) # Compute on the train data
X_b_full2 = np.c_[np.ones((X_test.shape[0], 1)), X_test2] # Predictions on the test / unseen data!
y_test_pred_closed_form2 = X_b_full2.dot(theta_best_full2)

# Train using Gradient Descent on full dataset
gd_model2 = LinearRegressionGD(learning_rate=0.00001, n_iterations=500000)  
gd_model2.fit(X_train2, y_train) # Compute on the train data
y_test_pred_gd2 = gd_model2.predict(X_test2) # Predictions on the test / unseen data!

# Convert predictions to pandas Series with y_test's original index
y_test_pred_closed_form2 = pd.Series(y_test_pred_closed_form2, index=y_test.index)
y_test_pred_gd2 = pd.Series(y_test_pred_gd2, index=y_test.index)

In [ ]:
# Create a figure with three subplots side by side
fig, axs = plt.subplots(1, 3, figsize=(18, 6))  # 1 row, 3 columns

# Subplot 1: Cost function over iterations (for Gradient Descent)
axs[0].plot(gd_model2.cost_history, label='Cost Function (MSE) over Iterations', color='b')
axs[0].set_xlabel('Iterations')
axs[0].set_ylabel('Mean Squared Error (MSE)')
axs[0].set_title('Gradient Descent Progress')
axs[0].legend()

# Subplot 2: Actual vs Predicted values for both methods
axs[1].plot(y_test, label='Actual Values', marker='o', color='g')
axs[1].plot(y_test_pred_gd2, label='Predicted Values (Gradient Descent)', marker='x', linestyle='--', color='r')
axs[1].plot(y_test_pred_closed_form2, label='Predicted Values (Closed Form)', marker='.', linestyle=':', color='b')
axs[1].set_xlabel('Sample Index')
axs[1].set_ylabel('Target Variable (y)')
axs[1].set_title('Actual vs Predicted Values (Test Set)')
axs[1].legend()

# Subplot 3: Parity plot for both methods (Predicted vs Actual with 45-degree line)
min_val = min(y_test.min(), y_test_pred_gd2.min(), y_test_pred_closed_form2.min())
max_val = max(y_test.max(), y_test_pred_gd2.max(), y_test_pred_closed_form2.max())

# Add the 45-degree perfect prediction line
axs[2].plot([min_val, max_val], [min_val, max_val], label='Perfect Prediction (45° Line)', linestyle='-', linewidth=4 ,color='g')
# Scatter plot for Gradient Descent predictions
axs[2].scatter(y_test, y_test_pred_gd2, label='Gradient Descent', color='r', alpha=0.4)
# Scatter plot for Closed-Form predictions
axs[2].scatter(y_test, y_test_pred_closed_form2, label='Closed Form', color='b', alpha=0.4)

axs[2].set_xlim(-0.2,9)
axs[2].set_xlabel('Actual Values')
axs[2].set_ylabel('Predicted Values')
axs[2].set_title('Parity Plot (Predicted vs Actual)')
axs[2].legend()

# Adjust layout for better spacing
plt.tight_layout()
plt.show()

# Output the coefficients for both methods
print("Gradient Descent Coefficients:", gd_model2.theta)
print("Closed-Form Solution Coefficients:", theta_best_full2)

# Compare the predictions and check if they are close
print("Are the predictions close? ", np.allclose(y_test_pred_gd2, y_test_pred_closed_form2))

In [ ]:
# Step 3.3: Evaluate the models 
rmse_gd2 = np.sqrt(mean_squared_error(y_test, y_test_pred_gd2))
r2_gd2 = r2_score(y_test, y_test_pred_gd2)
print(f'Mean Squared Error (Gradient Descent): {rmse_gd2}')
print(f'R-squared (Gradient Descent): {r2_gd2}')

rmse_closed_form2 = np.sqrt(mean_squared_error(y_test, y_test_pred_closed_form2))
r2_closed_form2 = r2_score(y_test, y_test_pred_closed_form2)
print(f'Mean Squared Error (Closed Form): {rmse_closed_form2}')
print(f'R-squared (Closed Form): {r2_closed_form2}')

In [ ]:
# Get coefficients from the Linear Regression model
cols = ['Intercept'] + list(X_train2.columns)
lr_coefficients_df2 = pd.DataFrame({ 'Feature': cols, 'Coefficient (GD)': gd_model2.theta, 'Coefficient (Closed Form)': theta_best_full2})
lr_coefficients_df2.T

## Step 4: Non-linear Regression

Step 4.1: Non-linear Regression through feature modification

Step 4.2: Non-linear Regression through weight modification

In [ ]:
# 4.1: Create polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Train the regression model on polynomial features
model_poly = LinearRegression()
model_poly.fit(X_train_poly, y_train)
y_pred_poly = model_poly.predict(X_test_poly)

################################################################################################################################################

# Step 4.2: Define random weights and fit the weighted linear regression model
weights = np.random.rand(X_train.shape[0])  # Random weights for demonstration -> how to set up 

# Train the regression model with specified weights
model_weighted = LinearRegression()
model_weighted.fit(X_train, y_train, sample_weight=weights)
y_pred_weighted = model_weighted.predict(X_test)

Step 4.3: Evaluation of the models with R^2 & RMSE

In [ ]:
# Step 4.3: Evaluation metrics for all models
rmse_linear = np.sqrt(mean_squared_error(y_test, y_test_pred_gd))
r2_linear = r2_score(y_test, y_test_pred_gd)
rmse_poly = np.sqrt(mean_squared_error(y_test, y_pred_poly))
r2_poly = r2_score(y_test, y_pred_poly)
rmse_weighted = np.sqrt(mean_squared_error(y_test, y_pred_weighted))
r2_weighted = r2_score(y_test, y_pred_weighted)

# Print the evaluation metrics
print(f"Linear Model R-Squared: {r2_linear}")
print(f"Polynomial Model R-Squared: {r2_poly}")
print(f"Weighted Model R-Squared: {r2_weighted}")

# Convert predictions to pandas Series with y_test's original index
y_pred_poly = pd.Series(y_pred_poly, index=y_test.index)
y_pred_weighted = pd.Series(y_pred_weighted, index=y_test.index)

In [ ]:
# Create a figure with six subplots (3x2): One row for actual vs. predicted, another for parity plots
fig, axs = plt.subplots(2, 3, figsize=(18, 12))  # 2 rows, 3 columns

### First row: Actual vs. Predicted values
# Subplot 1: Actual vs Predicted for Linear Regression (from Step 3)
axs[0, 0].plot(y_test, label='Actual Values', marker='o', color='g')
axs[0, 0].plot(y_test_pred_gd, label='Predicted (Linear)', marker='x', linestyle='--', color='b')
axs[0, 0].set_xlabel('Sample Index')
axs[0, 0].set_ylabel('Target Variable (y)')
axs[0, 0].set_title('Linear Regression')
axs[0, 0].legend()

# Subplot 2: Actual vs Predicted for Polynomial Regression (Step 4.1)
axs[0, 1].plot(y_test, label='Actual Values', marker='o', color='g')
axs[0, 1].plot(y_pred_poly, label='Predicted (Polynomial)', marker='x', linestyle='--', color='r')
axs[0, 1].set_xlabel('Sample Index')
axs[0, 1].set_ylabel('Target Variable (y)')
axs[0, 1].set_title('Polynomial Regression (Degree 2)')
axs[0, 1].legend()

# Subplot 3: Actual vs Predicted for Weighted Regression (Step 4.2)
axs[0, 2].plot(y_test, label='Actual Values', marker='o', color='g')
axs[0, 2].plot(y_pred_weighted, label='Predicted (Weighted)', marker='x', linestyle='--', color='orange')
axs[0, 2].set_xlabel('Sample Index')
axs[0, 2].set_ylabel('Target Variable (y)')
axs[0, 2].set_title('Weighted Regression')
axs[0, 2].legend()

### Second row: Parity plots (Predicted vs Actual, with 45-degree line)
min_val = min(y_test.min(), y_test_pred_gd.min(), y_pred_poly.min(), y_pred_weighted.min())
max_val = max(y_test.max(), y_test_pred_gd.max(), y_pred_poly.max(), y_pred_weighted.max())

# Subplot 4: Parity plot for Linear Regression
axs[1, 0].scatter(y_test, y_test_pred_gd, color='b', alpha=0.5, label='Linear Regression')
# axs[1, 0].plot([min_val, max_val], [min_val, max_val], 'k--', label='Perfect Prediction')
axs[1, 0].plot([min_val, max_val], [min_val, max_val], label='Perfect Prediction', linestyle='-', linewidth=4 ,color='g')
axs[1, 0].set_xlabel('Actual Values')
axs[1, 0].set_ylabel('Predicted Values')
axs[1, 0].set_title('Linear Regression Parity Plot')
axs[1, 0].set_xlim(-0.2,9)
axs[1, 0].legend()

# Subplot 5: Parity plot for Polynomial Regression
axs[1, 1].scatter(y_test, y_pred_poly, color='r', alpha=0.5, label='Polynomial Regression')
# axs[1, 1].plot([min_val, max_val], [min_val, max_val], 'k--', label='Perfect Prediction')
axs[1, 1].plot([min_val, max_val], [min_val, max_val], label='Perfect Prediction', linestyle='-', linewidth=4 ,color='g')
axs[1, 1].set_xlabel('Actual Values')
axs[1, 1].set_ylabel('Predicted Values')
axs[1, 1].set_title('Polynomial Regression Parity Plot')
axs[1, 1].set_xlim(-0.2,9)
axs[1, 1].legend()

# Subplot 6: Parity plot for Weighted Regression
axs[1, 2].scatter(y_test, y_pred_weighted, color='orange', alpha=0.5, label='Weighted Regression')
# axs[1, 2].plot([min_val, max_val], [min_val, max_val], 'k--', label='Perfect Prediction')
axs[1, 2].plot([min_val, max_val], [min_val, max_val], label='Perfect Prediction', linestyle='-', linewidth=4 ,color='g')
axs[1, 2].set_xlabel('Actual Values')
axs[1, 2].set_ylabel('Predicted Values')
axs[1, 2].set_title('Weighted Regression Parity Plot')
axs[1, 2].set_xlim(-0.2,9)
axs[1, 2].legend()

# Adjust layout for better spacing
plt.tight_layout()
plt.show()


## Step 5: Lasso + Ridge Regression

##### Lasso + Ridge (Package)

In [ ]:
# Initialize Lasso with alpha (regularization strength) parameter 
lasso = Lasso(alpha=0.1)
lasso.fit(X_train2, y_train) # including Wind Current
y_pred_lasso = lasso.predict(X_test2)

# Evaluate the Lasso model
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))
r2_lasso = r2_score(y_test, y_pred_lasso)
print(f"Root Mean Squared Error (RMSE): {rmse_lasso}")
print(f"R-squared (R²): {r2_lasso}")

# Get coefficients (non-zero ones indicate selected features)
print("#Lasso Coefficients != 0:", np.sum(lasso.coef_ != 0))
print("Lasso Coefficients:", lasso.coef_)

In [ ]:
# Convert predictions to pandas Series with y_test's original index
y_pred_lasso = pd.Series(y_pred_lasso, index=y_test.index)

# Plot
plt.figure(figsize=(18, 6))
plt.plot(y_test, label='Actual Values', color='g', alpha=0.5)
plt.plot(y_pred_lasso, label='Predicted Values (Lasso)', color='r') # linestyle='--',
plt.plot(y_test_pred_closed_form, label='Predicted Values (Closed Form)', color='b') # linestyle=':',
plt.xlabel('Sample Index')
plt.ylabel('Target Variable (y)')
plt.title('Actual vs Predicted Values (Test Set)')
plt.legend()
plt.xlim(6720, 8410)
plt.show()

LASSO with Cross-Validation to select optimal alpha value

In [ ]:
# Define a range of alpha values to test
alpha_values = np.logspace(-4, 0, num=20)

# Set up the Lasso model with CV
lasso2 = Lasso()
param_grid = {'alpha': alpha_values}
grid_search = GridSearchCV(estimator=lasso2, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best alpha
best_alpha = grid_search.best_params_['alpha']
print(f"Optimal Alpha: {best_alpha}")

In [ ]:
# Lasso Model
lasso3 = Lasso(alpha=best_alpha)
lasso3.fit(X_train, y_train)
y_pred_lasso = lasso3.predict(X_test)

# Evaluate Lasso model
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))
r2_lasso = r2_score(y_test, y_pred_lasso)
print(f"Root Mean Squared Error (RMSE): {rmse_lasso}")
print(f"R-squared (R²): {r2_lasso}")

# Coefficients
print("#Lasso Coefficients != 0:", np.sum(lasso3.coef_ != 0))
lasso_coefficients_df = pd.DataFrame({ 'Feature': X_train.columns, 'Coefficient': lasso3.coef_}) 
lasso_coefficients_df = lasso_coefficients_df[lasso_coefficients_df['Coefficient'] != 0] # Filter non-zeros
lasso_coefficients_df.T

In [ ]:
# Step 5.5: Analyze how regularization affects Lasso coefficients

from sklearn.linear_model import LinearRegression

# Train baseline Linear Regression for coefficient comparison
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
baseline_coefs = pd.Series(linear_model.coef_, index=X_train.columns, name="Baseline Linear Regression Coefs")

# Compare baseline vs Lasso coefficients
lasso_coefs = pd.Series(lasso3.coef_, index=X_train.columns, name="Lasso Coefs (Optimal Alpha)")
lasso_comparison = pd.concat([baseline_coefs, lasso_coefs], axis=1)
lasso_comparison['Change (Lasso)'] = lasso_comparison['Lasso Coefs (Optimal Alpha)'] - lasso_comparison['Baseline Linear Regression Coefs']

print("Lasso Coefficients Comparison (showing features shrunk by regularization):")
print(lasso_comparison[lasso_comparison['Lasso Coefs (Optimal Alpha)'] == 0])  # Features set to zero by Lasso
print("\nTop features with significant change under Lasso regularization:")
print(lasso_comparison[lasso_comparison['Change (Lasso)'].abs() > 0.1])  # Adjust threshold based on feature scale


In [ ]:
# Step 5.5: Analyze how regularization affects Lasso coefficients

from sklearn.linear_model import LinearRegression

# Train baseline Linear Regression for coefficient comparison
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
baseline_coefs = pd.Series(linear_model.coef_, index=X_train.columns, name="Baseline Linear Regression Coefs")

# Compare baseline vs Lasso coefficients
lasso_coefs = pd.Series(lasso3.coef_, index=X_train.columns, name="Lasso Coefs (Optimal Alpha)")
lasso_comparison = pd.concat([baseline_coefs, lasso_coefs], axis=1)
lasso_comparison['Change (Lasso)'] = lasso_comparison['Lasso Coefs (Optimal Alpha)'] - lasso_comparison['Baseline Linear Regression Coefs']

print("Lasso Coefficients Comparison (showing features shrunk by regularization):")
print(lasso_comparison[lasso_comparison['Lasso Coefs (Optimal Alpha)'] == 0])  # Features set to zero by Lasso
print("\nTop features with significant change under Lasso regularization:")
print(lasso_comparison[lasso_comparison['Change (Lasso)'].abs() > 0.1])  # Adjust threshold based on feature scale


RIDGE

In [ ]:
from sklearn.linear_model import Ridge

# Build Ridge model
ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_train)

# Evaluate Ridge model
rmse_ridge = np.sqrt(mean_squared_error(y_test_reg, y_pred))
mae_ridge = mean_absolute_error(y_test_reg, y_pred)
r2_ridge = r2_score(y_test_reg, y_pred)

print(f"Root Mean Squared Error (RMSE): {rmse_ridge}")
print(f"Mean Absolute Error (MAE): {mae_ridge}")
print(f"R-squared (R²): {r2_ridge}")

# Coefficients
print("#Ridge Coefficients != 0:", np.sum(ridge.coef_ != 0))
# Save coefficients
theta_best_ridge = np.append(ridge.intercept_, ridge.coef_)
ridge_coefficients_df = pd.DataFrame({ 'Feature': X_train_reg.columns, 'Coefficient': ridge.coef_})
ridge_coefficients_df = ridge_coefficients_df[ridge_coefficients_df['Coefficient'] != 0] # Filter non-zeros
ridge_coefficients_df.T

In [ ]:
def ridge_closed_form_solution(X, y, alpha=1.0):
    # Ensure X and y are NumPy arrays and convert them to float type
    X = np.array(X, dtype=float)
    y = np.array(y, dtype=float)

    # Add a bias (intercept) column of ones to the input matrix X
    X_b = np.c_[np.ones((X.shape[0], 1)), X]

    # Normal equation: theta = (X.T * X + alpha * I)^(-1) * X.T * y
    identity_matrix = np.eye(X_b.shape[1])
    theta_best = np.linalg.inv(X_b.T.dot(X_b) + alpha * identity_matrix).dot(X_b.T).dot(y)

    return theta_best

In [ ]:
# Step 5.5: Analyze how regularization affects Ridge coefficients

# Get Ridge coefficients and compare them to baseline
ridge_coefs = pd.Series(ridge.coef_, index=X_train.columns, name="Ridge Coefs (Optimal Alpha)")
ridge_comparison = pd.concat([baseline_coefs, ridge_coefs], axis=1)
ridge_comparison['Change (Ridge)'] = ridge_comparison['Ridge Coefs (Optimal Alpha)'] - ridge_comparison['Baseline Linear Regression Coefs']

print("\nRidge Coefficients Comparison (highlighting features most affected):")
print(ridge_comparison[ridge_comparison['Change (Ridge)'].abs() > 0.1])  # Adjust threshold as needed


### Step 5.1: Linear Regression with Regularization

Implementation of Linear Regression with Regularization

In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin

class LinearRegressionWithRegularization(BaseEstimator, RegressorMixin):
    def __init__(self, learning_rate=0.01, n_iterations=1000, alpha=0.1, regularization="l1"):
        # Hyperparameters
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.alpha = alpha  # Regularization strength
        self.regularization = regularization
        self.theta = None  # Parameters (weights and bias)

    def fit(self, X, y):
        # Ensure X and y are NumPy arrays
        X = np.array(X, dtype=float)
        y = np.array(y, dtype=float)

        # Add a bias (intercept) column of ones to the input matrix X
        X_b = np.c_[np.ones((X.shape[0], 1)), X]  # X_b now includes the bias term
        m, n = X_b.shape

        # Initialize theta
        self.theta = np.zeros(n)

        # Gradient Descent loop
        for _ in range(self.n_iterations):
            # Predicted values (y_hat = X_b * theta)
            y_pred = X_b.dot(self.theta)

            # Compute the gradient
            gradient = (1 / m) * X_b.T.dot(y_pred - y)

            # Apply regularization to the gradient (skip bias term)
            if self.regularization == "l2":
                # L2 Regularization (Ridge): (MSE + L2 penalty)
                gradient[1:] += 2 * self.alpha * self.theta[1:]
            elif self.regularization == "l1":
                # L1 Regularization (LASSO): (MSE + L1 penalty)
                for j in range(1, n):
                    if self.theta[j] > 0:
                        gradient[j] += self.alpha
                    elif self.theta[j] < 0:
                        gradient[j] -= self.alpha

            # Update parameters
            self.theta -= self.learning_rate * gradient

    def predict(self, X):
        # Ensure X is a NumPy array and convert it to float type
        X = np.array(X, dtype=float)

        # Add a bias (intercept) column of ones to the input matrix X
        X_b = np.c_[np.ones((X.shape[0], 1)), X]

        # Return predictions (y_pred = X_b * theta)
        return X_b.dot(self.theta)

In [ ]:
# Test model for L1 (LASSO)
model_l1 = LinearRegressionWithRegularization(alpha=0.05, regularization="l1")
model_l1.fit(X_train, y_train)
y_pred_l1 = model_l1.predict(X_test)

# Evaluate L1 model
r2_l1 = r2_score(y_test, y_pred_l1)
print(f"R-squared (Lasso): {r2_l1}")

# Test model for L2 (RIDGE)
model_l2 = LinearRegressionWithRegularization(alpha=0.05, regularization="l2")
model_l2.fit(X_train, y_train)
y_pred_l2 = model_l2.predict(X_test)

# Evaluate L2 model
r2_l2 = r2_score(y_test, y_pred_l2)
print(f"R-squared (Ridge): {r2_l2}")

# Convert predictions to pandas Series with y_test's original index
y_pred_l1 = pd.Series(y_pred_l1, index=y_test.index)
y_pred_l2 = pd.Series(y_pred_l2, index=y_test.index)

Try to find the optimal alpha value with Parameter GridSearch

In [ ]:
# Initialize the model
model_lasso = LinearRegressionWithRegularization(regularization="l1", learning_rate=0.001, n_iterations=10000)
# Define a range of alpha values to test
alpha_values = np.logspace(-4, 0, num=20)
param_grid = {'alpha': alpha_values}

# Set up the Lasso model with CV
grid_search = GridSearchCV(estimator=model_lasso, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best alpha
best_alpha = grid_search.best_params_['alpha']
print(f"Optimal Alpha: {best_alpha}")

# Best model after grid search
best_model = grid_search.best_estimator_
y_pred_l1 = best_model.predict(X_test)

# Evaluate the model
r2_l1 = r2_score(y_test, y_pred_l1)

# Coefficients
print("#Lasso Coefficients != 0:", np.sum(best_model.theta != 0))
cols = ['Intercept'] + list(X_train.columns)
lasso_coefficients_df = pd.DataFrame({ 'Feature': cols, 'Coefficient': best_model.theta})
lasso_coefficients_df = lasso_coefficients_df[lasso_coefficients_df['Coefficient'] != 0] # Filter non-zeros
lasso_coefficients_df.T

### Step 5.2: Non-Linear Regression with Regularization

In [ ]:
class PolynomialRegressionWithRegularization(BaseEstimator, RegressorMixin):
    def __init__(self, degree, learning_rate=0.01, n_iterations=1000, alpha=0.1, regularization="l1"):
        # Hyperparameters
        self.degree = degree
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.alpha = alpha # = Regularization strength
        self.regularization = regularization
        self.theta = None

    def _polynomial_features(self, X):
        # Generate polynomial features up to specified degree
        X_poly = np.ones((X.shape[0], 1)) # Bias term
        for d in range(1, self.degree + 1):
            X_poly = np.c_[X_poly, X ** d]
        return X_poly

    # Gradient Descent Fit
    def fit(self, X, y):
        # Generate polynomial features
        X_poly = self._polynomial_features(X)
        # Number of training samples (m) and number of features (n)
        m, n = X_poly.shape
        # Initialize theta
        self.theta = np.zeros(n)
        
        # Gradient Descent loop
        for _ in range(self.n_iterations):
            # Predicted values (y_hat = X_b * theta)
            y_pred = X_poly.dot(self.theta)
            
            # Compute the gradient
            gradient = (1 / m) * X_poly.T.dot(y_pred - y)
            
            # Apply regularization to the gradient (& skip bias term)
            if self.regularization == "l2":
                # L2 Regularization (Ridge): (MSE + L2 penalty)
                gradient[1:] += 2 * self.alpha * self.theta[1:]
            elif self.regularization == "l1":
                # L1 Regularization (LASSO): (MSE + L1 penalty)
                for j in range(1, n):
                    if self.theta[j] > 0:
                        gradient[j] += self.alpha
                    elif self.theta[j] < 0:
                        gradient[j] -= self.alpha
            
            # Update parameters
            self.theta -= self.learning_rate * gradient
    
    def predict(self, X):
        # Generate polynomial features (including bias term)
        X_poly = self._polynomial_features(X)
        
        # Return predictions
        return X_poly.dot(self.theta)

DEGREE=3 doesnt work / loads forever!!!!

In [ ]:
# Test model for L1 (LASSO)
model_nl1 = PolynomialRegressionWithRegularization(degree=2, alpha=0.05, learning_rate=0.001, n_iterations=10000, regularization="l1")
model_nl1.fit(X_train, y_train)
y_pred_nl1 = model_nl1.predict(X_test)

# Evaluate L1 model
r2_nl1 = r2_score(y_test, y_pred_nl1)
print(f"R-squared (Lasso): {r2_nl1}")

# Test model for L2 (RIDGE)
model_nl2 = PolynomialRegressionWithRegularization(degree=2, alpha=0.05, learning_rate=0.001, n_iterations=10000, regularization="l2")
model_nl2.fit(X_train, y_train)
y_pred_nl2 = model_nl2.predict(X_test)

# Evaluate L2 model
r2_nl2 = r2_score(y_test, y_pred_nl2)
print(f"R-squared (Ridge): {r2_nl2}")

# Convert predictions to pandas Series with y_test's original index
y_pred_nl1 = pd.Series(y_pred_nl1, index=y_test.index)
y_pred_nl2 = pd.Series(y_pred_nl2, index=y_test.index)

## Step 6?: Revenue Calculation

In [ ]:
# Convert y_pred_lasso to df
y_pred_lasso_df = pd.DataFrame(y_pred_lasso, columns=['y_pred_lasso'])
# Get ts for test
ts_test = ts[split_index:]

# Merge them on index
pred = pd.merge(ts_test, y_pred_lasso_df, left_index=True, right_index=True)

In [ ]:
# Convert y_test_pred_gd to df
y_test_pred_gd_df = pd.DataFrame(y_test_pred_gd, columns=['y_pred_gd'])
# Convert y_pred_poly to df
y_pred_poly_df = pd.DataFrame(y_pred_poly, columns=['y_pred_poly'])
# Convert y_pred_weighted to df
y_pred_weighted_df = pd.DataFrame(y_pred_weighted, columns=['y_pred_weighted'])
# Convert y_pred_l1 to df
y_pred_l1_df = pd.DataFrame(y_pred_l1, columns=['y_pred_l1'])
# Convert y_pred_nl1 to df
y_pred_nl1_df = pd.DataFrame(y_pred_nl1, columns=['y_pred_nl1'])
# Convert y_pred_l2 to df
y_pred_l2_df = pd.DataFrame(y_pred_l2, columns=['y_pred_l2'])
# Convert y_pred_nl2 to df
y_pred_nl2_df = pd.DataFrame(y_pred_nl2, columns=['y_pred_nl2'])
# Get ts for test
ts_test = ts[split_index:]

# Merge them on index
preds = pd.merge(ts_test, y_test_pred_gd_df, left_index=True, right_index=True)
preds = pd.merge(preds, y_pred_poly_df, left_index=True, right_index=True)
preds = pd.merge(preds, y_pred_weighted_df, left_index=True, right_index=True)
preds = pd.merge(preds, y_pred_l1_df, left_index=True, right_index=True)
preds = pd.merge(preds, y_pred_nl1_df, left_index=True, right_index=True)
preds = pd.merge(preds, y_pred_l2_df, left_index=True, right_index=True)
preds = pd.merge(preds, y_pred_nl2_df, left_index=True, right_index=True)
preds.head()

In [ ]:
nordpool.head()

In [ ]:
energinet.head()